In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score,train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score,average_precision_score

1.Загрузка данных и препроцессинг

In [2]:
data=pd.read_csv('orange_small_churn_train_data.csv')
data.shape

(40000, 232)

In [3]:
data.head()

,ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [4]:
print 'Доля оттока',(sum([1. for l in data['labels'] if l == 1])/len(data))
print 'Доля не оттока',(-sum([-1. for l in data['labels'] if l == -1])/len(data))

Доля оттока 0.0744
Доля не оттока 0.9256


In [5]:
data=data.drop('ID',1)

In [6]:
X_data=data.iloc[:,:230]
y_data=data.iloc[:,230:]

In [7]:
dataT=pd.read_csv('orange_small_churn_test_data.csv')
dataT.shape

(10000, 231)

In [8]:
X_dataT=dataT.drop('ID',1)

In [9]:
def preproceccing(X1,X2,number_features):
    #сливаем вместе чтобы провести единообразный препроцессинг и кодирование категорий
    X=pd.concat([X_data,X_dataT])
    print X.shape
    #Отбросим колонки где данных вообще нет
    zero=list()
    s=X.describe()
    for var in s.columns:
        if s[var]['count']==0:zero.append(var)
    print zero
    print 'Колонки ,которые мы отбросили', len(zero)
    real_columns=X.columns.drop(zero)
    X=X[real_columns]
    print X.shape
    #Из 230 колонок выбросли уже 18 где нет данных(из них 16 вещ. и 2 кат.).190(последняя вещ.)-16=174
    #разделим на вещ. и категориальные и заполним вещ.пропуски средними,а пропуски категории "NA"
    num_data=X.iloc[:,:174]
    cat_data=X.iloc[:,174:]
    num_data_mean=num_data.fillna(num_data.describe().loc['mean'])
    cat_data=cat_data.fillna('NA')
    print num_data.shape
    print cat_data.shape
    print y_data.shape
    #Мы видели что есть кат признаки с кол-вом категорий более 10000.Возможно это фамилии или персональные данные,
    #которые сейчас нам будут только мешать.Проверим есть ли различия по кол-ву категорий в признаке в обучающей и 
    #тестовой выборке
    k=0
    for var in cat_data.columns:
        for name in list(cat_data.iloc[40000:,:][var].value_counts().index):
            if name not in list(cat_data.iloc[:40000,:][var].value_counts().index): k+=1
        print var,k
        k=0
    #Ясно что эти признаки надо убрать, будут только шуметь.Оставим только только признаки с числом категорий меньше
    #number_features
    k=0
    new=list()
    for var in cat_data.columns:
        if len(cat_data[var].value_counts())<number_features : 
            k+=1
            print 'X_train',var,len(cat_data.iloc[:40000,:][var].value_counts()),'   X_test',var,len(cat_data.iloc[40000:,:][var].value_counts())
            new.append(var)
    print new
    print 'Признаки которые мы оставили',k
    cat=cat_data[new]
    #Кодируем их
    cat_code = cat.apply(LabelEncoder().fit_transform)
    cat_code = OneHotEncoder(sparse=False).fit_transform(cat_code)
    print cat_code.shape
    #Разделяем выборки заново
    cat_codeL=cat_code[:40000,:]
    cat_codeT=cat_code[40000:,:]
    num_data_meanL=num_data_mean.iloc[:40000,:]
    num_data_meanT=num_data_mean.iloc[40000:,:]
    #Масштабируем признаки
    num_data_meanL=StandardScaler().fit_transform(num_data_meanL)
    num_data_meanT=StandardScaler().fit_transform(num_data_meanT)
    #Собираем признаки вместе
    X_selectL=np.hstack( (num_data_meanL,cat_codeL))# обучение
    X_selectT=np.hstack( (num_data_meanT,cat_codeT))# тестовая без меток
    print X_selectL.shape
    
    return  X_selectL,X_selectT 

In [10]:
X_selectL, X_selectT = preproceccing(X_data,X_dataT,number_features=1000)

(50000, 230)
['Var8', 'Var15', 'Var20', 'Var31', 'Var32', 'Var39', 'Var42', 'Var48', 'Var52', 'Var55', 'Var79', 'Var141', 'Var167', 'Var169', 'Var175', 'Var185', 'Var209', 'Var230']
Колонки ,которые мы отбросили 18
(50000, 212)
(50000, 174)
(50000, 38)
(40000, 1)
X_train Var191 2    X_test Var191 2
X_train Var192 355    X_test Var192 280
X_train Var193 50    X_test Var193 41
X_train Var194 4    X_test Var194 4
X_train Var195 23    X_test Var195 17
X_train Var196 4    X_test Var196 3
X_train Var197 221    X_test Var197 184
X_train Var201 3    X_test Var201 2
X_train Var203 6    X_test Var203 5
X_train Var204 100    X_test Var204 100
X_train Var205 4    X_test Var205 4
X_train Var206 22    X_test Var206 22
X_train Var207 14    X_test Var207 11
X_train Var208 3    X_test Var208 3
X_train Var210 6    X_test Var210 6
X_train Var211 2    X_test Var211 2
X_train Var212 78    X_test Var212 67
X_train Var213 2    X_test Var213 2
X_train Var215 2    X_test Var215 2
X_train Var218 3    X_test Var

In [11]:
X, X_holdout, y, y_holdout = train_test_split(X_selectL, np.array(y_data).reshape(40000,), 
                                              test_size=0.3, random_state=0, stratify=y_data)
print X.shape,X_holdout.shape
print y.shape,y_holdout.shape

(28000L, 1200L) (12000L, 1200L)
(28000L,) (12000L,)


2.Обучение.
Я выбрал LogisticRegression и GradientBoostingClassifier,бустинг после балансировки дал более высокие уровни метрик.

In [12]:
#Выбираем тип разбиения для кросс-валидации
#Делаем кроссвалидацию по F-мере,ROC-AUC,PRC
cv=StratifiedKFold(n_splits=3)

In [17]:
clf=GradientBoostingClassifier(n_estimators=50,random_state=1)
CV=cross_val_score(clf,X,y,scoring = 'f1',cv=cv)
CV1=cross_val_score(clf,X,y,scoring = 'average_precision',cv=cv)
CV2=cross_val_score(clf,X,y,scoring = 'roc_auc',cv=cv)
print 'f1               ',CV.mean()
print 'average_precision',CV1.mean()
print 'roc_auc          ',CV2.mean()

f1                0.013252868091
average_precision 0.193256497567
roc_auc           0.72973463646


In [13]:
#F1-мера явно низкая
#Сбалансируем классы досэмплитровав объекты минорного класса
np.random.seed(1)
indices_to_add = np.random.randint(np.sum(y==1),size=(np.sum(y==-1)-np.sum(y==1)))
X_a =X[y == 1,:][indices_to_add,:]
X_add=np.vstack( (X,X_a) )
ones= np.ones((np.sum(y==-1)-np.sum(y==1)))
y_add=np.hstack((y, ones))
y_add.shape

(51834L,)

In [20]:
clf=GradientBoostingClassifier(n_estimators=50,random_state=1)
CV=cross_val_score(clf,X_add,y_add,scoring = 'f1',cv=cv)
CV1=cross_val_score(clf,X_add,y_add,scoring = 'average_precision',cv=cv)
CV2=cross_val_score(clf,X_add,y_add,scoring = 'roc_auc',cv=cv)
print 'f1               ',CV.mean()
print 'average_precision',CV1.mean()
print 'roc_auc          ',CV2.mean()

f1                0.697924418976
average_precision 0.751002701911
roc_auc           0.766427675931


In [23]:
#уже все получше.Пробуем обучить и проверить на отложенной выборке
gb=GradientBoostingClassifier(n_estimators=50,random_state=1)
gb.fit(X_add, y_add)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=1,
              subsample=1.0, verbose=0, warm_start=False)

In [24]:
gb_predictions=gb.predict_proba(X_holdout)
gb_predictions1=gb.predict(X_holdout)

In [23]:
auc=roc_auc_score(y_holdout,gb_predictions[:,1]  ) 
print(classification_report(y_holdout,gb_predictions1 ) )
f1=f1_score(y_holdout,gb_predictions1)
avr=average_precision_score(y_holdout,gb_predictions1)
print 'f1               ',f1
print 'average_precision',avr
print 'roc_auc          ',auc

             precision    recall  f1-score   support

         -1       0.96      0.68      0.80     11107
          1       0.15      0.69      0.24       893

avg / total       0.90      0.68      0.75     12000

f1                0.242686039662
average_precision 0.431054397963
roc_auc           0.745956541434


Попробуем подбирать параметры

In [14]:
number=[50, 100, 150]
for n in number:
    clf = GradientBoostingClassifier(n_estimators=n,random_state=1)
    auc=cross_val_score(clf, X_add, y_add, cv=cv, scoring='roc_auc')
    print n, auc.mean()

50 0.766427675931
100 0.792403682581
150 0.810539567256


In [15]:
number=[1, 5, 10]
for n in number:
    clf = GradientBoostingClassifier(min_samples_leaf=n,random_state=1)
    auc=cross_val_score(clf, X_add, y_add, cv=cv, scoring='roc_auc')
    print n, auc.mean()

1 0.792403682581
5 0.791600375571
10 0.791618756794


In [16]:
number=[3, 5, 7]
for n in number:
    clf = GradientBoostingClassifier(max_depth=n,random_state=1)
    auc=cross_val_score(clf, X_add, y_add, cv=cv, scoring='roc_auc')
    print n, auc.mean()

3 0.792403682581
5 0.863068307898
7 0.928121992883


In [ ]:
#есть подозрение что мы переобучились

In [17]:
gb=GradientBoostingClassifier(n_estimators=100,min_samples_leaf=1,max_depth=1,random_state=1)
gb.fit(X_add, y_add)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=150, presort='auto', random_state=1,
              subsample=1.0, verbose=0, warm_start=False)

In [18]:
gb_predictions=gb.predict_proba(X_holdout)
gb_predictions1=gb.predict(X_holdout)

In [19]:
auc=roc_auc_score(y_holdout,gb_predictions[:,1]  ) 
print(classification_report(y_holdout,gb_predictions1 ) )
f1=f1_score(y_holdout,gb_predictions1)
avr=average_precision_score(y_holdout,gb_predictions1)
print 'f1               ',f1
print 'average_precision',avr
print 'roc_auc          ',auc

             precision    recall  f1-score   support

         -1       0.96      0.76      0.85     11107
          1       0.16      0.56      0.25       893

avg / total       0.90      0.75      0.80     12000

f1                0.248069738481
average_precision 0.375050285975
roc_auc           0.726938037623


In [25]:
gb_predictions1=gb.predict_proba(X_selectT)

In [26]:
rez=pd.DataFrame(data=gb_predictions1[:,1], columns=['result'])
rez.to_csv('rez3.csv',index=True, index_label='Id')

In [ ]:
#На kaggle загрузил вероятности по классу 1,результат в скриншоте 0.7270 ,что выше бэнчмарка